<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/label.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/프로젝트/final project/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
nv = pd.read_excel(f"{DATA_PATH}naver_reviews_1.xlsx")
nv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19580 entries, 0 to 19579
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        19580 non-null  object
 1   review    19117 non-null  object
 2   date      19580 non-null  object
 3   date_cnt  19580 non-null  int64 
 4   keyword   17944 non-null  object
dtypes: int64(1), object(4)
memory usage: 765.0+ KB


In [ ]:
df = nv.copy()

In [ ]:
df.isnull().sum()

id             0
review       463
date           0
date_cnt       0
keyword     1636
dtype: int64

In [ ]:
# NaN 값이 있는 행 제거
df.dropna(subset=['keyword', 'review'], inplace=True)

# 빈 문자열인 행 제거
df = df[df['review'].str.strip() != ""]

# 공백 제거
df['review'] = df['review'].str.strip()

# 라인 끝의 '\n' 제거
df['review'] = df['review'].str.replace('\n', ' ')

# 한글과 숫자를 제외한 모든 문자 제거
pattern = r'[^0-9가-힣\s]'
df['review'] = df['review'].str.replace(pattern, '', regex=True)

# 'keyword'와 'review' 열이 공백인 행을 제거
df = df[(df['keyword'] != '') & (df['review'] != '')]

<ipython-input-6-446b4f4b0a05>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].str.strip()
<ipython-input-6-446b4f4b0a05>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].str.replace('\n', ' ')
<ipython-input-6-446b4f4b0a05>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [ ]:
df.isnull().sum()

id          0
review      0
date        0
date_cnt    0
keyword     0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17342 entries, 0 to 19579
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        17342 non-null  object
 1   review    17342 non-null  object
 2   date      17342 non-null  object
 3   date_cnt  17342 non-null  int64 
 4   keyword   17342 non-null  object
dtypes: int64(1), object(4)
memory usage: 812.9+ KB


In [ ]:
# 'review' 열에서 한글로 세글자 이하인 내용을 가진 행 삭제
df = df[df['review'].str.len() > 6]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14840 entries, 0 to 19579
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        14840 non-null  object
 1   review    14840 non-null  object
 2   date      14840 non-null  object
 3   date_cnt  14840 non-null  int64 
 4   keyword   14840 non-null  object
dtypes: int64(1), object(4)
memory usage: 695.6+ KB


In [ ]:
df['review']

0                          마드레생산구이정식 맛있고 반찬 많아서  외식메뉴로 좋아요
1        정말 맛있어요 반찬도 다양해서 너무 좋아요 최고특히 생선구이가 제일인것 같아요 불고...
2             부모님 모시고 오기 딱 좋은 음식점이예요 반찬부터 생선구이 모든게 다 맛있습니다
3         맛은 쏘쏘인데 자반정식 2인분 3만2000원인데 한마리1인분에 반마리인 셈 나오는건 좀
4        예쁜 꽃이 반겨주고 맛난밥이 기분좋게해주는  근데 어버이날 특수여서 간장게장을 먹을...
                               ...                        
19575                                   아주아주 맛있습니다 동백카츠 최고
19576                                            깔끔하고 맛있어요
19577    첫 방문 항상 사람 많아 못 들어왔는데 오늘은 대기없이 바로 들어올수 있어 굿 맛 ...
19578                                           직원분들이 친절해요
19579                                   친구랑 너무 맛있게 먹고 왔습니다
Name: review, Length: 14840, dtype: object

In [ ]:
import random

random_samples = df.sample(n=5)

for index, row in random_samples.iterrows():
    print(f"Review: {row['review']}\nKeywords: {row['keyword']}\n---")


Review: 엄마가 발사믹을 안 좋아하시는데도 발사믹 관자는 잘 드셨습니다 그만큼 부담스러운 맛이 아니라 정말 관자랑 잘 어울리는 조화로운  맛이었고 위에 올라간 오징어채와 아래의 감자 퓨레도 맛있었습니다 새우 리조또도 시켰는데 새우가 큼직하고 먹물 리조또라서 빨간  새우  퓨레  까만 밥의 색감 조화도 음식을 눈으로도 보는 것 같았습니다 시그니처인 멜팅 치즈 케이크도 최고였어요 엄마는 
Keywords: 매장이 넓어요,인테리어가 멋져요,음식이 맛있어요,특별한 메뉴가 있어요,친절해요
---
Review: 명불허전 수내역까지 체인내셨네요 축하드립니다
Keywords: 혼밥하기 좋아요,음식이 맛있어요,특별한 메뉴가 있어요,재료가 신선해요,친절해요
---
Review: 양도 많고 맛이 좋네요
Keywords: 음식이 맛있어요
---
Review: 소갈비솥밥을 먹었는데 갈비가 질기지않고 식감도 좋았어요 인테리어도 예쁘고 양도 충분한것같았어요 다음에 또 와서 다른메뉴도 먹어보고싶네요 잘먹고 갑니다 
Keywords: 음식이 맛있어요,특별한 메뉴가 있어요,인테리어가 멋져요
---
Review: 크리스마스를 기념해서 지인과 함께 방문했습니다 음식의 퀄리티도 좋았고 매장 분위기도 좋았습니다 단지 매장이 조금 협소하고 예 약자분들이 많으셔서 그런가 코스로 나와야할 요리들이 거의 한번에 다나와서 조금 놀라긴했습니다 다음에 이용하게 된다면 조금 한 가한 시간에 매장 안쪽에서 이용하고싶은 마음이네요  잘먹었습니다
Keywords: 음식이 맛있어요,가성비가 좋아요,특별한 날 가기 좋아요
---


In [ ]:
df.isnull().sum()

id          0
review      0
date        0
date_cnt    0
keyword     0
dtype: int64

In [ ]:
all_keywords = []
for keyword_set in df['keyword']:
    all_keywords.extend(keyword_set.split(','))

all_keywords

['음식이 맛있어요',
 '양이 많아요',
 '음식이 맛있어요',
 '양이 많아요',
 '가성비가 좋아요',
 '친절해요',
 '음식이 맛있어요',
 '인테리어가 멋져요',
 '주차하기 편해요',
 '음식이 맛있어요',
 '인테리어가 멋져요',
 '특별한 날 가기 좋아요',
 '친절해요',
 '매장이 청결해요',
 '뷰가 좋아요',
 '차분한 분위기에요',
 '음식이 맛있어요',
 '건강한 맛이에요',
 '반찬이 잘 나와요',
 '단체모임 하기 좋아요',
 '차분한 분위기에요',
 '음식이 맛있어요',
 '단체모임 하기 좋아요',
 '양이 많아요',
 '음식이 맛있어요',
 '주차하기 편해요',
 '음식이 맛있어요',
 '재료가 신선해요',
 '특별한 메뉴가 있어요',
 '가성비가 좋아요',
 '매장이 넓어요',
 '음식이 맛있어요',
 '친절해요',
 '뷰가 좋아요',
 '음식이 맛있어요',
 '주차하기 편해요',
 '특별한 날 가기 좋아요',
 '음식이 맛있어요',
 '친절해요',
 '음식이 맛있어요',
 '단체모임 하기 좋아요',
 '음식이 맛있어요',
 '주차하기 편해요',
 '인테리어가 멋져요',
 '대화하기 좋아요',
 '컨셉이 독특해요',
 '반찬이 잘 나와요',
 '매장이 넓어요',
 '친절해요',
 '주차하기 편해요',
 '특별한 날 가기 좋아요',
 '음식이 맛있어요',
 '재료가 신선해요',
 '양이 많아요',
 '주차하기 편해요',
 '음식이 맛있어요',
 '뷰가 좋아요',
 '단체모임 하기 좋아요',
 '주차하기 편해요',
 '가성비가 좋아요',
 '음식이 맛있어요',
 '친절해요',
 '주차하기 편해요',
 '음식이 맛있어요',
 '특별한 메뉴가 있어요',
 '양이 많아요',
 '음식이 맛있어요',
 '특별한 메뉴가 있어요',
 '단체모임 하기 좋아요',
 '뷰가 좋아요',
 '음식이 맛있어요',
 '매장이 청결해요',
 '음식이 맛있어요',
 '친절해요',
 '주차하기 편해요',
 '음식이 맛있어요',
 '음식이 맛있어요',
 '친절

In [ ]:
from collections import Counter

# all_keywords 리스트에 중복된 키워드의 개수를 세기
keyword_counter = Counter(all_keywords)

# 빈도수가 높은 순으로 키워드 출력
for keyword, count in keyword_counter.most_common():
    print(f"{keyword}: {count}")


음식이 맛있어요: 9769
친절해요: 4932
특별한 메뉴가 있어요: 3719
재료가 신선해요: 3601
매장이 청결해요: 3200
인테리어가 멋져요: 2935
가성비가 좋아요: 2419
매장이 넓어요: 2381
양이 많아요: 2311
혼밥하기 좋아요: 1574
디저트가 맛있어요: 1124
특별한 날 가기 좋아요: 1033
주차하기 편해요: 842
 음식이 맛있어요: 817
 재료가 신선해요: 758
뷰가 좋아요: 751
 친절해요: 718
단체모임 하기 좋아요: 686
커피가 맛있어요: 604
 가성비가 좋아요: 595
 양이 많아요: 521
 매장이 청결해요: 510
 특별한 메뉴가 있어요: 493
음료가 맛있어요: 397
대화하기 좋아요: 380
사진이 잘 나와요: 342
 매장이 넓어요: 330
 인테리어가 멋져요: 309
 주차하기 편해요: 285
화장실이 깨끗해요: 281
고기 질이 좋아요: 241
좌석이 편해요: 205
메뉴 구성이 알차요: 202
음식이 빨리 나와요: 171
 단체모임 하기 좋아요: 147
포장이 깔끔해요: 138
현지 맛에 가까워요: 132
빵이 맛있어요: 119
 화장실이 깨끗해요: 104
차분한 분위기에요: 100
 혼밥하기 좋아요: 96
비싼 만큼 가치있어요: 95
 특별한 날 가기 좋아요: 89
컨셉이 독특해요: 81
음악이 좋아요: 81
 고기 질이 좋아요: 81
 사진이 잘 나와요: 76
아늑해요: 73
 대화하기 좋아요: 69
집중하기 좋아요: 67
 뷰가 좋아요: 62
건강한 맛이에요: 54
 디저트가 맛있어요: 54
직접 잘 구워줘요: 52
 좌석이 편해요: 42
 기본 안주가 좋아요: 41
술이 다양해요: 39
아이와 가기 좋아요: 38
오래 머무르기 좋아요: 30
 음료가 맛있어요: 29
 술이 다양해요: 29
야외 공간이 멋져요: 26
기본 안주가 좋아요: 25
 아늑해요: 25
 메뉴 구성이 알차요: 24
 커피가 맛있어요: 23
선물하기 좋아요: 20
반찬이 잘 나와요: 19
 건강한 맛이에요: 17
종

In [ ]:
# 공백 제거
all_keywords = [keyword.strip() for keyword in all_keywords]

# 중복 제거
unique_keywords = list(set(all_keywords))


In [ ]:
len(unique_keywords)

52

In [ ]:
unique_keywords

['재료가 신선해요',
 '디저트가 맛있어요',
 '선물하기 좋아요',
 '좌석이 편해요',
 '오래 머무르기 좋아요',
 '혼술하기 좋아요',
 '비싼 만큼 가치있어요',
 '친절해요',
 '매장이 넓어요',
 '아늑해요',
 '음료가 맛있어요',
 '주차하기 편해요',
 '차가 맛있어요',
 '사진이 잘 나와요',
 '커피가 맛있어요',
 '포장이 깔끔해요',
 '집중하기 좋아요',
 '특별한 메뉴가 있어요',
 '매장이 청결해요',
 '반려동물과 가기 좋아요',
 '야외 공간이 멋져요',
 '고기 질이 좋아요',
 '기본 안주가 좋아요',
 '코스요리가 알차요',
 '건강한 맛이에요',
 '빵이 맛있어요',
 '혼밥하기 좋아요',
 '화장실이 깨끗해요',
 '인테리어가 멋져요',
 '단체모임 하기 좋아요',
 '직접 잘 구워줘요',
 '음악이 좋아요',
 '현지 맛에 가까워요',
 '술이 다양해요',
 '환기가 잘 돼요',
 '대화하기 좋아요',
 '음식이 맛있어요',
 '특별한 날 가기 좋아요',
 '컨셉이 독특해요',
 '뷰가 좋아요',
 '아이와 가기 좋아요',
 '잡내가 적어요',
 '반찬이 잘 나와요',
 '양이 많아요',
 '종류가 다양해요',
 '차분한 분위기에요',
 '음식이 빨리 나와요',
 '가성비가 좋아요',
 '룸이 잘 되어있어요',
 '메뉴 구성이 알차요',
 '향신료가 강하지 않아요',
 '샐러드바가 잘 되어있어요']

In [ ]:
sorted(unique_keywords)

['가성비가 좋아요',
 '건강한 맛이에요',
 '고기 질이 좋아요',
 '기본 안주가 좋아요',
 '단체모임 하기 좋아요',
 '대화하기 좋아요',
 '디저트가 맛있어요',
 '룸이 잘 되어있어요',
 '매장이 넓어요',
 '매장이 청결해요',
 '메뉴 구성이 알차요',
 '반려동물과 가기 좋아요',
 '반찬이 잘 나와요',
 '뷰가 좋아요',
 '비싼 만큼 가치있어요',
 '빵이 맛있어요',
 '사진이 잘 나와요',
 '샐러드바가 잘 되어있어요',
 '선물하기 좋아요',
 '술이 다양해요',
 '아늑해요',
 '아이와 가기 좋아요',
 '야외 공간이 멋져요',
 '양이 많아요',
 '오래 머무르기 좋아요',
 '음료가 맛있어요',
 '음식이 맛있어요',
 '음식이 빨리 나와요',
 '음악이 좋아요',
 '인테리어가 멋져요',
 '잡내가 적어요',
 '재료가 신선해요',
 '종류가 다양해요',
 '좌석이 편해요',
 '주차하기 편해요',
 '직접 잘 구워줘요',
 '집중하기 좋아요',
 '차가 맛있어요',
 '차분한 분위기에요',
 '친절해요',
 '커피가 맛있어요',
 '컨셉이 독특해요',
 '코스요리가 알차요',
 '특별한 날 가기 좋아요',
 '특별한 메뉴가 있어요',
 '포장이 깔끔해요',
 '향신료가 강하지 않아요',
 '현지 맛에 가까워요',
 '혼밥하기 좋아요',
 '혼술하기 좋아요',
 '화장실이 깨끗해요',
 '환기가 잘 돼요']

In [ ]:
remove_keywords = [
    "반려동물과 가기 좋아요", "뷰가 좋아요", "선물하기 좋아요", "사진이 잘 나와요",
    "야외 공간이 멋져요", "음악이 좋아요", "인테리어가 멋져요", "좌석이 편해요", "현지 맛에 가까워요",
    "화장실이 깨끗해요", "환기가 잘 돼요", "매장이 청결해요", "매장이 넓어요", "단체모임하기 좋아요", "주차하기 편해요", "특별한 메뉴가 있어요", "특별한 날 가기 좋아요", "뷰가 좋아요", "차분한 분위기에요"
]


In [ ]:
# 지정한 키워드를 각 행의 'keyword' 열에서 제거하는 함수
def remove_specific_keywords_from_row(row):
    row_keywords = row.split(',')
    row_keywords = [keyword.strip() for keyword in row_keywords]  # 공백 제거
    filtered_keywords = [keyword for keyword in row_keywords if keyword not in remove_keywords]
    return ', '.join(filtered_keywords)

# 특정 키워드 제거
df['keyword'] = df['keyword'].apply(remove_specific_keywords_from_row)

# 모든 키워드를 콤마와 공백을 기준으로 다시 분리
all_keywords_after_removal = []
for keyword_combo in df['keyword']:
    all_keywords_after_removal.extend(keyword_combo.split(','))

# 공백 제거
all_keywords_after_removal = [keyword.strip() for keyword in all_keywords_after_removal]

# 중복 제거
unique_keywords_after_removal = list(set(all_keywords_after_removal))

# 'keyword'와 'review' 열이 공백인 행을 제거
df = df[(df['keyword'] != '') & (df['review'] != '')]


# 삭제된 키워드가 있는지 확인
remaining_keywords_intersect = set(unique_keywords_after_removal) & set(remove_keywords)

len(unique_keywords_after_removal), sorted(unique_keywords_after_removal), remaining_keywords_intersect


(36,
 ['',
  '가성비가 좋아요',
  '건강한 맛이에요',
  '고기 질이 좋아요',
  '기본 안주가 좋아요',
  '단체모임 하기 좋아요',
  '대화하기 좋아요',
  '디저트가 맛있어요',
  '룸이 잘 되어있어요',
  '메뉴 구성이 알차요',
  '반찬이 잘 나와요',
  '비싼 만큼 가치있어요',
  '빵이 맛있어요',
  '샐러드바가 잘 되어있어요',
  '술이 다양해요',
  '아늑해요',
  '아이와 가기 좋아요',
  '양이 많아요',
  '오래 머무르기 좋아요',
  '음료가 맛있어요',
  '음식이 맛있어요',
  '음식이 빨리 나와요',
  '잡내가 적어요',
  '재료가 신선해요',
  '종류가 다양해요',
  '직접 잘 구워줘요',
  '집중하기 좋아요',
  '차가 맛있어요',
  '친절해요',
  '커피가 맛있어요',
  '컨셉이 독특해요',
  '코스요리가 알차요',
  '포장이 깔끔해요',
  '향신료가 강하지 않아요',
  '혼밥하기 좋아요',
  '혼술하기 좋아요'],
 set())

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14160 entries, 0 to 19579
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        14160 non-null  object
 1   review    14160 non-null  object
 2   date      14160 non-null  object
 3   date_cnt  14160 non-null  int64 
 4   keyword   14160 non-null  object
dtypes: int64(1), object(4)
memory usage: 663.8+ KB


In [ ]:
import random

random_samples = df.sample(n=5)

for index, row in random_samples.iterrows():
    print(f"Review: {row['review']}\nKeywords: {row['keyword']}\n---")


Review: 깔끔한 분위기 맛있는 커피 너무 좋아요
Keywords: 커피가 맛있어요, 디저트가 맛있어요, 친절해요
---
Review: 마제소바와 치즈모둠카츠세트를 주문했어요 음식이 정갈하고 정말 맛있어요 
Keywords: 혼밥하기 좋아요, 음식이 맛있어요
---
Review: 집 근처라 가깝고 맛있어서 좋아요
Keywords: 혼밥하기 좋아요, 음식이 맛있어요, 재료가 신선해요, 친절해요
---
Review: 웨이팅 줄이 생각보다 빨리 빠져요 기다리면서 지나다니는 사람들 때문에 정신없긴한데 자리에 앉으면 걍 맛있고 재미
Keywords: 음식이 맛있어요
---
Review: 언니가 추천해서 처음 가봤는데 가성비 좋고 맛도 있어서 자주 방문할듯 해요 오픈이벤트 중이라 라구파스타까지 알차게 잘 먹었네요  맛있게 잘 먹었어요
Keywords: 음식이 맛있어요, 가성비가 좋아요
---


In [ ]:
!pip install transformers

#합성 데이터 생성(예시)

In [ ]:
# word_mapping = {
#     '맛있어요': '맛없어요',
#     '좋아요': '나빠요',
#     '친절해요': '불친절해요',
#     '편해요': '불편해요',
#     '멋져요': '별로에요',
#     '깔끔해요': '지저분해요',
#     '많아요': '적어요'
# }

# def generate_reverse_reviews(review, word_mapping):
#     reverse_review = review
#     reverse_keywords = []

#     for positive_word, negative_word in word_mapping.items():
#         if positive_word in reverse_review:
#             reverse_review = reverse_review.replace(positive_word, negative_word)
#             reverse_keywords.append(negative_word)

#     return reverse_review, ','.join(reverse_keywords)


In [ ]:
# test_review = "음식이 맛있어요, 서비스도 좋아요, 분위기가 멋져요"
# generate_reverse_reviews(test_review, word_mapping)

In [ ]:
# reverse_rows = []

# # 반전 리뷰 생성
# for index, row in df.iterrows():
#     review = row['review']
#     if not isinstance(review, str):  # 리뷰 문자열 x -> 스킵합니다.
#         continue

#     reverse_review, reverse_keywords = generate_reverse_reviews(review, word_mapping)
#     if reverse_keywords:
#         reverse_rows.append({
#             'review': reverse_review,
#             'keyword': reverse_keywords
#         })


In [ ]:
# reverse_df = pd.DataFrame(reverse_rows)

In [ ]:
# reverse_df.head(20)

# 토큰화

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# 키워드 분리
df['keyword'] = df['keyword'].apply(lambda x: [k.strip() for k in x.split(',')])

# MultiLabelBinarizer 원핫 인코딩
mlb = MultiLabelBinarizer()
encoded_labels = mlb.fit_transform(df['keyword'])
label_classes = mlb.classes_

# 토크나이저 및 모델 로딩
from transformers import AutoTokenizer, AutoModel
model_name = "bongsoo/bert-base-kor-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 리뷰 토큰화
train_texts = df['review'].tolist()
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)

# UNK 토큰으로 1인 값 0으로 치환
import numpy as np
input_ids = np.array(train_encodings['input_ids'])
for i in range(input_ids.shape[0]):
    input_ids[i][input_ids[i] == 1] = 0


In [ ]:
encoded_labels

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
label_classes

array(['가성비가 좋아요', '건강한 맛이에요', '고기 질이 좋아요', '기본 안주가 좋아요', '단체모임 하기 좋아요',
       '대화하기 좋아요', '디저트가 맛있어요', '룸이 잘 되어있어요', '메뉴 구성이 알차요', '반찬이 잘 나와요',
       '비싼 만큼 가치있어요', '빵이 맛있어요', '샐러드바가 잘 되어있어요', '술이 다양해요', '아늑해요',
       '아이와 가기 좋아요', '양이 많아요', '오래 머무르기 좋아요', '음료가 맛있어요', '음식이 맛있어요',
       '음식이 빨리 나와요', '잡내가 적어요', '재료가 신선해요', '종류가 다양해요', '직접 잘 구워줘요',
       '집중하기 좋아요', '차가 맛있어요', '친절해요', '커피가 맛있어요', '컨셉이 독특해요', '코스요리가 알차요',
       '포장이 깔끔해요', '향신료가 강하지 않아요', '혼밥하기 좋아요', '혼술하기 좋아요'], dtype=object)

In [ ]:
# 원핫인코딩된 레이블 확인
print("원핫인코딩된 레이블의 형태:", encoded_labels.shape)
print("첫 5개의 행의 원핫 인코딩된 레이블:", encoded_labels[:5])
print("레이블 클래스:", label_classes)

# 토큰화 확인
print("리뷰 개수:", len(train_texts))
print("원본 텍스트 예시:", train_texts[0])
print("토큰화된 input_ids 예시:", train_encodings['input_ids'][0])
print("토큰화된 attention_mask 예시:", train_encodings['attention_mask'][0])

# input_ids 길이 확인
print("첫 번째 input_ids의 길이:", len(train_encodings['input_ids'][0]))


원핫인코딩된 레이블의 형태: (14160, 35)
첫 5개의 행의 원핫 인코딩된 레이블: [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]]
레이블 클래스: ['가성비가 좋아요' '건강한 맛이에요' '고기 질이 좋아요' '기본 안주가 좋아요' '단체모임 하기 좋아요' '대화하기 좋아요'
 '디저트가 맛있어요' '룸이 잘 되어있어요' '메뉴 구성이 알차요' '반찬이 잘 나와요' '비싼 만큼 가치있어요' '빵이 맛있어요'
 '샐러드바가 잘 되어있어요' '술이 다양해요' '아늑해요' '아이와 가기 좋아요' '양이 많아요' '오래 머무르기 좋아요'
 '음료가 맛있어요' '음식이 맛있어요' '음식이 빨리 나와요' '잡내가 적어요' '재료가 신선해요' '종류가 다양해요'
 '직접 잘 구워줘요' '집중하기 좋아요' '차가 맛있어요' '친절해요' '커피가 맛있어요' '컨셉이 독특해요' '코스요리가 알차요'
 '포장이 깔끔해요' '향신료가 강하지 않아요' '혼밥하기 좋아요' '혼술하기 좋아요']
리뷰 개수: 14160
원본 텍스트 예시: 마드레생산구이정식 맛있고 반찬 많아서  외식메뉴로 좋아요
토큰화된 input_ids 예시: [2, 8758, 1105, 1339, 1051, 24210, 14479, 6700, 1133, 11023, 398, 1223, 1107, 5567, 1499,

# 데이터셋 및 데이터 로더 설정

# 모델 설정

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertForSequenceClassification, AdamW
import torch.nn as nn

# DataLoader 설정
train_dataset = TensorDataset(
    torch.tensor(train_encodings['input_ids'], dtype=torch.long),
    torch.tensor(train_encodings['attention_mask'], dtype=torch.long),
    torch.tensor(encoded_labels, dtype=torch.float)
)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# 모델 불러오기
num_labels = encoded_labels.shape[1]
model = BertForSequenceClassification.from_pretrained("bongsoo/bert-base-kor-v1", num_labels=num_labels)
model = model.to(device)

# 손실 함수와 옵티마이저 설정
criterion = nn.BCEWithLogitsLoss()

# 수정된 코드
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bongsoo/bert-base-kor-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 검증
print(f"라벨 번호: {num_labels}")
print(f"Dataloader 첫 배치: {next(iter(train_loader))}")
print(f"Model device: {next(model.parameters()).device}")


라벨 번호: 35
Dataloader 첫 배치: [tensor([[    2,  4004,  1791,  ...,     0,     0,     0],
        [    2,  2073,  6367,  ...,     0,     0,     0],
        [    2, 10029,  2650,  ...,     0,     0,     0],
        ...,
        [    2,  8070,  1606,  ...,     0,     0,     0],
        [    2,   645,  1252,  ...,     0,     0,     0],
        [    2,     1,  6700,  ...,     0,     0,     0]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

# 모델학습

In [ ]:
from tqdm import tqdm
import time
import os
from transformers import AdamW, get_linear_schedule_with_warmup
import torch

epochs = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)

total_steps = len(train_loader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

save_path = "/content/drive/MyDrive/"

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")

    model.train()

    epoch_start_time = time.time()
    total_loss = 0

    for i, batch in tqdm(enumerate(train_loader), total=len(train_loader)):
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

        outputs = model(input_ids, attention_mask=attention_mask)  # labels=labels 를 제거
        logits = outputs.logits  # 로짓 가져오기

        loss = criterion(logits, labels.type_as(logits))  # BCEWithLogitsLoss 사용

        total_loss += loss.item()

        loss.backward()


        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 파라미터 업데이트
        optimizer.step()

        # learning rate 조정
        scheduler.step()

        # 그라디언트 초기화
        optimizer.zero_grad()

    avg_train_loss = total_loss / len(train_loader)

    epoch_end_time = time.time()
    epoch_time_elapsed = epoch_end_time - epoch_start_time

    print(f"  평균 학습 loss: {avg_train_loss}")
    print(f"  에폭 토크 학습: {epoch_time_elapsed}s")

    # 저장
    model_save_path = os.path.join(save_path, f"model_epoch_{epoch+1}")
    tokenizer_save_path = os.path.join(save_path, f"tokenizer_epoch_{epoch+1}")
    model.save_pretrained(model_save_path)
    tokenizer.save_pretrained(tokenizer_save_path)

    print(f"  에폭 저장 {epoch+1}")

print("완료")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/2


100%|██████████| 885/885 [06:05<00:00,  2.42it/s]


  평균 학습 loss: 0.15834127102194534
  에폭 토크 학습: 365.9197552204132s
  에폭 저장 1
Epoch 2/2


 83%|████████▎ | 734/885 [05:03<01:02,  2.40it/s]

In [ ]:
# from sklearn.metrics import f1_score, accuracy_score
# import numpy as np
# import torch
# from torch.utils.data import DataLoader, TensorDataset
# from transformers import BertForSequenceClassification
# import torch.nn as nn

# def permutation_train_test_split(input_ids, attention_mask, target, test_size=0.2, shuffle=True, random_state=SEED):  # SEED 값
#     test_num = int(input_ids.shape[0] * test_size)
#     train_num = input_ids.shape[0] - test_num

#     if shuffle:
#         np.random.seed(random_state)
#         shuffled = np.random.permutation(input_ids.shape[0])

#         input_ids = input_ids[shuffled,:]
#         attention_mask = attention_mask[shuffled,:]
#         target = target[shuffled,:]

#         train_input_ids = input_ids[:train_num]
#         train_attention_mask = attention_mask[:train_num]

#         test_input_ids = input_ids[train_num:]
#         test_attention_mask = attention_mask[train_num:]

#         train_target = target[:train_num]
#         test_target = target[train_num:]

#     else:
#         train_input_ids = input_ids[:train_num]
#         train_attention_mask = attention_mask[:train_num]

#         test_input_ids = input_ids[train_num:]
#         test_attention_mask = attention_mask[train_num:]

#         train_target = target[:train_num]
#         test_target = target[train_num:]

#     return train_input_ids, train_attention_mask, test_input_ids, test_attention_mask, train_target, test_target



In [ ]:
# 훈련 데이터와 테스트 데이터 나누기
train_input_ids, train_attention_mask, test_input_ids, test_attention_mask, train_target, test_target = permutation_train_test_split(
    np.array(train_encodings['input_ids']),
    np.array(train_encodings['attention_mask']),
    np.array(encoded_labels),
    test_size=0.2,
    shuffle=True,
    random_state= SEED
)



# 테스트 데이터를 DataLoader에 적용
test_dataset = TensorDataset(
    torch.tensor(test_input_ids, dtype=torch.long),
    torch.tensor(test_attention_mask, dtype=torch.long),
    torch.tensor(test_target, dtype=torch.float)
)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# 모델 평가
model.eval()
test_loss = 0
all_preds = []
all_labels = []

for batch in test_loader:
    input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = criterion(logits, labels.type_as(logits))

    test_loss += loss.item()
    preds = torch.sigmoid(logits)
    all_preds.extend(preds.cpu().numpy())
    all_labels.extend(labels.cpu().numpy())

# 평가 지표 계산
test_loss /= len(test_loader)
predicted_labels = np.round(np.array(all_preds))
accuracy = accuracy_score(np.array(all_labels), predicted_labels)

# 1번: Micro-average F1 Score
f1_micro = f1_score(np.array(all_labels), predicted_labels, average='micro')

# 2번: Macro-average F1 Score
f1_macro = f1_score(np.array(all_labels), predicted_labels, average='macro')

# 3번: Weighted-average F1 Score
f1_weighted = f1_score(np.array(all_labels), predicted_labels, average='weighted')

# 4번: Sample-average F1 Score
f1_samples = f1_score(np.array(all_labels), predicted_labels, average='samples')


print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {accuracy}")
print(f"F1 Score (Micro): {f1_micro}") # 모든 라벨 동등하게 취급(빈도가 낮은 클래스는 높은 클래스에 의해 영향 크게받음)
print(f"F1 Score (Macro): {f1_macro}") #라벨 간 상관관계가 높지 않을 때(모든 라벨 동등하게 취급, 드물게 나타나는 라벨 고려)
print(f"F1 Score (Weighted): {f1_weighted}") #라벨이 불균형할 때 라벨의 중요도 고려(라벨의 빈도수 가중평균)
print(f"F1 Score (Samples): {f1_samples}")  # 각 샘플에 대한 라벨 예측의 정확도를 고려



In [ ]:
# import numpy as np
# import torch
# from torch import nn, optim
# from torch.utils.data import DataLoader, TensorDataset
# from sklearn.metrics import f1_score
# from sklearn.model_selection import KFold

# # 임의의 데이터 생성 (input_ids, attention_mask, labels)
# num_samples = 1000
# num_classes = 5
# input_ids = np.random.randint(0, 100, size=(num_samples, 512))
# attention_mask = np.random.randint(0, 2, size=(num_samples, 512))
# labels = np.random.randint(0, 2, size=(num_samples, num_classes))

# # 모델 정의
# class SimpleModel(nn.Module):
#     def __init__(self):
#         super(SimpleModel, self).__init__()
#         self.fc1 = nn.Linear(512, 128)
#         self.fc2 = nn.Linear(128, num_classes)

#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.fc2(x)
#         return x

# # 모델, optimizer, loss 초기화
# model = SimpleModel()
# optimizer = optim.Adam(model.parameters())
# criterion = nn.BCEWithLogitsLoss()

# # K-Fold와 threshold 설정
# kf = KFold(n_splits=5)
# thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]

# # 교차 검증 및 threshold 테스트
# for threshold in thresholds:
#     f1_scores = []

#     for train_idx, test_idx in kf.split(input_ids):
#         train_dataset = TensorDataset(
#             torch.tensor(input_ids[train_idx], dtype=torch.float),
#             torch.tensor(attention_mask[train_idx], dtype=torch.float),
#             torch.tensor(labels[train_idx], dtype=torch.float)
#         )
#         train_loader = DataLoader(train_dataset, batch_size=32)

#         test_dataset = TensorDataset(
#             torch.tensor(input_ids[test_idx], dtype=torch.float),
#             torch.tensor(attention_mask[test_idx], dtype=torch.float),
#             torch.tensor(labels[test_idx], dtype=torch.float)
#         )
#         test_loader = DataLoader(test_dataset, batch_size=32)

#         # 학습
#         model.train()
#         for batch in train_loader:
#             batch_input_ids, batch_attention_mask, batch_labels = batch
#             optimizer.zero_grad()
#             outputs = model(batch_input_ids)
#             loss = criterion(outputs, batch_labels)
#             loss.backward()
#             optimizer.step()

#         # 평가
#         model.eval()
#         all_preds = []
#         all_labels = []

#         with torch.no_grad():
#             for batch in test_loader:
#                 batch_input_ids, batch_attention_mask, batch_labels = batch
#                 outputs = model(batch_input_ids)
#                 all_preds.extend(torch.sigmoid(outputs).cpu().numpy())
#                 all_labels.extend(batch_labels.cpu().numpy())

#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)

#         # 예측 라벨 결정
#         predicted_labels = (all_preds > threshold).astype(int)

#         # F1 Score 계산
#         f1 = f1_score(all_labels, predicted_labels, average='micro')
#         f1_scores.append(f1)

#     # 결과 출력
#     avg_f1 = np.mean(f1_scores)
#     print(f"Threshold: {threshold}, Avg F1 Score: {avg_f1}")


In [ ]:
yogi = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_jsi.csv")

In [ ]:
yogi.head()

In [ ]:
new_reviews= yogi["고객리뷰"]

In [ ]:
new_reviews

In [ ]:
from tqdm.auto import tqdm

def batch_predict(model, input_ids, attention_mask, batch_size):
    model.eval()
    num_batches = len(input_ids) // batch_size + int(len(input_ids) % batch_size != 0)
    probabilities = []
    for batch_num in tqdm(range(num_batches), desc="Predicting"):
        batch_start = batch_num * batch_size
        batch_end = (batch_num + 1) * batch_size
        batch_input_ids = input_ids[batch_start:batch_end].to(device)
        batch_attention_mask = attention_mask[batch_start:batch_end].to(device)
        with torch.no_grad():
            outputs = model(batch_input_ids, attention_mask=batch_attention_mask)
            logits = outputs.logits
            batch_probabilities = torch.sigmoid(logits).cpu()
            probabilities.append(batch_probabilities)
    return torch.cat(probabilities, dim=0)

In [ ]:
# 새로운 리뷰 데이터 토큰화
new_reviews_list = new_reviews.tolist()
new_encodings = tokenizer(new_reviews_list, truncation=True, padding=True, max_length=512)

input_ids = torch.tensor(new_encodings['input_ids'])
attention_mask = torch.tensor(new_encodings['attention_mask'])

# 평가 모드로 설정
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 예측 수행 (batch_predict 함수 사용)
probabilities = batch_predict(model, input_ids, attention_mask, batch_size=1)

# 예측된 라벨의 확률을 기반으로 실제 키워드 라벨을 결정
threshold = 0.5
predicted_indices = (probabilities > threshold).int()
predicted_labels = mlb.inverse_transform(predicted_indices.numpy())

for i, (review, labels) in enumerate(zip(new_reviews, predicted_labels)):
    print(f"리뷰: {review}")
    print(f"예측 라벨: {labels}")

In [ ]:
one_star = yogi[yogi["별점"] ==1 ]

In [ ]:
one_star["고객리뷰"]

In [ ]:
new_reviews_list = one_star["고객리뷰"]

In [ ]:

# 새로운 리뷰 데이터 토큰화
new_reviews_list = one_star["고객리뷰"].tolist()
new_encodings = tokenizer(new_reviews_list, truncation=True, padding=True, max_length=512)

input_ids = torch.tensor(new_encodings['input_ids'])
attention_mask = torch.tensor(new_encodings['attention_mask'])

# 모델을 평가 모드로 설정
model.eval()

# 모델과 입력 데이터를 동일한 디바이스로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 예측 수행 (batch_predict 함수 사용)
probabilities = batch_predict(model, input_ids, attention_mask, batch_size=1)

# 예측된 라벨의 확률을 기반으로 실제 키워드 라벨을 결정
threshold = 0.5
predicted_indices = (probabilities > threshold).int()
predicted_labels = mlb.inverse_transform(predicted_indices.numpy())

# 결과 출력
for i, (review, labels) in enumerate(zip(new_reviews, predicted_labels)):
    print(f"리뷰: {review}")
    print(f"예측 라벨: {labels}")

# ...


In [ ]:

# # 새로운 리뷰 데이터 토큰화
# new_reviews_list = one_star["고객리뷰"].tolist()
# new_encodings = tokenizer(new_reviews_list, truncation=True, padding=True, max_length=512)

# input_ids = torch.tensor(new_encodings['input_ids'])
# attention_mask = torch.tensor(new_encodings['attention_mask'])

# # 모델을 평가 모드로 설정
# model.eval()

# # 모델과 입력 데이터를 동일한 디바이스로 이동
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

# # 예측 수행 (batch_predict 함수 사용)
# probabilities = batch_predict(model, input_ids, attention_mask, batch_size=1)

# # 예측된 라벨의 확률을 기반으로 실제 키워드 라벨을 결정
# threshold = 0.5
# predicted_indices = (probabilities > threshold).int()
# predicted_labels = mlb.inverse_transform(predicted_indices.numpy())

# # 결과 출력
# for i, (review, labels) in enumerate(zip(new_reviews_list, predicted_labels)):
#     print(f"리뷰: {review}")
#     print(f"예측 라벨: {labels}")

# # ...


In [ ]:
# from skorch import NeuralNetBinaryClassifier
# import torch

# class BertClassifier(NeuralNetBinaryClassifier):
#     def __init__(self, *args, **kwargs):
#         super(BertClassifier, self).__init__(*args, **kwargs)

#     def initialize(self):
#         self.module_ = self.module(
#             num_labels=len(label_classes),
#         )
#         return self


In [ ]:
# from sklearn.metrics import f1_score
# from transformers import BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
# import torch
# from torch.utils.data import DataLoader, TensorDataset
# import numpy as np
# import time
# from torch.optim import AdamW
# from tqdm import tqdm  # tqdm 추가

# def train_and_evaluate_model(train_loader, test_loader, num_labels, device, epochs=1, learning_rate=2e-5, batch_size=16):
#     # 모델 초기화
#     model = BertForSequenceClassification.from_pretrained("bongsoo/bert-base-kor-v1", num_labels=num_labels)
#     model = model.to(device)

#     optimizer = AdamW(model.parameters(), lr=learning_rate)

#     # 총 훈련 스텝 계산
#     total_steps = len(train_loader) * epochs

#     # 스케줄러 설정
#     scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

#     # 손실 함수 설정
#     criterion = torch.nn.BCEWithLogitsLoss()

#     # 훈련 시작
#     for epoch in range(epochs):
#         model.train()
#         total_loss = 0
#         for i, batch in tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Training epoch {epoch+1}"):
#             input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)
#             outputs = model(input_ids, attention_mask=attention_mask)
#             loss = criterion(outputs.logits, labels.type_as(outputs.logits))
#             total_loss += loss.item()
#             loss.backward()
#             optimizer.step()
#             scheduler.step()
#             optimizer.zero_grad()

#         # 평가 시작
#         model.eval()
#         all_preds = []
#         all_labels = []
#         for i, batch in tqdm(enumerate(test_loader), total=len(test_loader), desc=f"Evaluating epoch {epoch+1}"):
#             input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)
#             with torch.no_grad():
#                 outputs = model(input_ids, attention_mask=attention_mask)
#                 logits = outputs.logits
#             preds = torch.sigmoid(logits)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())

#         # F1 스코어 계산
#         predicted_labels = np.round(np.array(all_preds))
#         f1 = f1_score(np.array(all_labels), predicted_labels, average='micro')
#         print(f"Epoch {epoch+1}/{epochs} - F1 Score: {f1}")

#     return f1
# # 하이퍼파라미터 그리드 설정
# param_grid = {
#     'epochs': [1, 2],
#     'learning_rate': [2e-5, 3e-5],
#     'batch_size': [16, 32]
# }

# # 최적 F1 스코어 저장 변수 초기화
# best_f1 = 0
# best_params = {}

# # 그리드 서치 시작
# for epochs in param_grid['epochs']:
#     for learning_rate in param_grid['learning_rate']:
#         for batch_size in param_grid['batch_size']:
#             print(f"Training with epochs={epochs}, learning_rate={learning_rate}, batch_size={batch_size}")

#             # DataLoader 설정
#             train_dataset = TensorDataset(
#                 torch.tensor(train_input_ids, dtype=torch.long),
#                 torch.tensor(train_attention_mask, dtype=torch.long),
#                 torch.tensor(train_target, dtype=torch.float)
#             )
#             train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

#             test_dataset = TensorDataset(
#                 torch.tensor(test_input_ids, dtype=torch.long),
#                 torch.tensor(test_attention_mask, dtype=torch.long),
#                 torch.tensor(test_target, dtype=torch.float)
#             )
#             test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#             # 모델 훈련 및 평가
#             f1 = train_and_evaluate_model(train_loader, test_loader, num_labels=len(label_classes), device=device, epochs=epochs, learning_rate=learning_rate, batch_size=batch_size)

#             # 최적 F1 스코어 업데이트
#             if f1 > best_f1:
#                 best_f1 = f1
#                 best_params = {'epochs': epochs, 'learning_rate': learning_rate, 'batch_size': batch_size}

# print(f"Best F1 Score: {best_f1}")
# print(f"Best Parameters: {best_params}")
